In [1]:
!pip install transformers datasets evaluate
!pip install accelerate -U
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


필요 패키지 설치

In [1]:
from huggingface_hub import notebook_login

notebook_login()

Huggingface 업로드
https://huggingface.co/BLACKBUN/Tesla_related_classification

In [2]:
import json

with open('/content/drive/MyDrive/combined_dict_list_lc.json', 'r') as f:
		data = json.load(f)

print(data[:5])

[{'label': 1, 'text': 'general motors to use tesla charging standard from 2025 from 2025, general motors will rely on tesla’s charging port in the usa and canada – exclusively. gm wants to provide adapters so that all cars equipped with the north american charging standard (nacs) can also charge at ccs stations. this makes gm the ...'}, {'label': 1, 'text': 'owners of gm evs will have access to tesla superchargers in 2024 just three months after tesla announced it would open its supercharger network to non-tesla electric vehicles — and only weeks after ford claimed the first spot in line — gm has announced a ...'}, {'label': 0, 'text': 'westwood exits radio show after sexual misconduct claims dj tim westwood leaves capital xtra "until further notice" after sexual misconduct allegations.'}, {'label': 0, 'text': "countries with homophobic laws unlikely to host the games' federation says human rights and inclusiveness are ingrained in its constitution."}, {'label': 1, 'text': 'tesla opens

In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

print(f"Number of elements in training set: {len(train_data)}")
print(f"Number of elements in testing set: {len(test_data)}")

Number of elements in training set: 34775
Number of elements in testing set: 3864


데이터셋 학습:테스트 = 9:1 설정

In [4]:
from datasets import Dataset, DatasetDict

# Convert the data into HuggingFace dataset
hf_train = Dataset.from_dict({k: [d[k] for d in train_data] for k in train_data[0]})
hf_test = Dataset.from_dict({k: [d[k] for d in test_data] for k in test_data[0]})

# Print the first five rows
print(hf_train[:5])
print(hf_test[:5])


{'label': [1, 1, 1, 1, 1], 'text': ['revamped tesla model_3 could be revealed as musk visits tesla’s shanghai factory according to bloomberg, tesla will showcase its updated model_3 on thursday when ceo elon musk visits tesla’s shanghai factory. bloomberg’s sources, who didn’t wish to be identified, said the first cars coming off the line on thursday will be ...', 'tesla’s china expansion hits speed bump amid industry overcapacity shanghai (reuters) -tesla’s ambitious plan to boost auto production in shanghai, its most valuable plant globally, hinges on china’s approvals to develop 70 hectares (172 acres) of former farmland that is currently overgrown with wildflowers.', '20 2019 tesla model_x for sale in canada only show cars that can be delivered to me. please enter your postal code in order to show cars that can be delivered to you.', 'tesla jumps as gm deal makes its charging network closer to us standard (reuters) -tesla shares closed 4% higher on friday after general motors joine

In [5]:
news_data = DatasetDict({
    'train': hf_train,
    'test': hf_test
})

Transformers 이용 위해 데이터 Dataset Class로 변환

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

new_tokens = ['tesla', 'model_s', 'model_3', 'model_x', 'model_y',
              'supercharger', 'cybertruck', 'general motors', 'gm', 'ford', 'elon', 'musk']

new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())

tokenizer.add_tokens(list(new_tokens))

9

토크나이저에 테슬라 관련 토큰 추가
모든 텍스트 소문자 가정, 'model 3'같은 경우 'model_3'로 전처리됨

In [9]:
def preprocess_function(examples):
		return tokenizer(examples["text"], truncation=True)

In [10]:
tokenized_news_data = news_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/34775 [00:00<?, ? examples/s]

Map:   0%|          | 0/3864 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

텍스트 토크나이징

In [12]:
import evaluate

accuracy = evaluate.load("accuracy")

In [13]:
import numpy as np

def compute_metrics(eval_pred):
		logits, labels = eval_pred
		predictions = np.argmax(logits, axis=1)
		return accuracy.compute(predictions=predictions, references=labels)

In [14]:
id2label = {0: "Unrelated", 1: "Related"}
label2id = {"Unrelated": 0, "Related": 1}

In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2,
                                                           id2label=id2label, label2id=label2id)

model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.we

Embedding(30531, 768)

학습 전 필요 요소 정의

In [17]:
training_args = TrainingArguments(
    output_dir="tesla-related-unrelated-classification",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_news_data["train"],
    eval_dataset=tokenized_news_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/BLACKBUN/tesla-related-unrelated-classification into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.001900,0.001843,0.999741
2,0.000000,0.000744,0.999741


TrainOutput(global_step=4348, training_loss=0.00423269104872245, metrics={'train_runtime': 586.9554, 'train_samples_per_second': 118.493, 'train_steps_per_second': 7.408, 'total_flos': 1420019725271148.0, 'train_loss': 0.00423269104872245, 'epoch': 2.0})

학습 및 결과 도출, 데이터 테스트셋 기준 정확도 99.9%

In [ ]:
text = "Tesla Model S &amp; Model X have hospital-grade HEPA air filters, protecting you from dust, bacteria, pollen (allergies), spores &amp; many toxic gases"

In [19]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model='BLACKBUN/tesla-related-unrelated-classification')
classifier("model_3 is great")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'Unrelated', 'score': 0.9999208450317383}]

In [20]:
from transformers import AutoTokenizer

text = "model_3 is doing great"

inputs = tokenizer(text, return_tensors='pt')

print(inputs)

{'input_ids': tensor([[  101, 30528,  2003,  2725,  2307,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
